# langchain backend -- lecture 3

In [36]:
# pip install -r requirements.txt

## test 00 langchain example

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import DirectoryLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub

In [15]:
import os
# get the current directory
os.getcwd()

'/Users/junjiezhang/Documents/GitHub/LLM_Zeng24/backend_langchain'

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import DirectoryLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub

# Loader for multiple text files
loader = DirectoryLoader('./', glob="*.txt")
documents = loader.load()
# Load all text files in the current directory

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)
# Split the loaded documents into chunks of 1000 characters with 200 characters overlap

# Create embeddings for each chunk
vectorstore = Chroma.from_documents(chunks, OpenAIEmbeddings())
# Generate embeddings for each chunk using OpenAI embeddings and store them in a vector database

# Define a retriever from the vector DB
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
# Create a retriever that will fetch the top 6 most similar chunks based on similarity search

# Get a prompt from LangChain hub
prompt = hub.pull("rlm/rag-prompt")
# Retrieve a predefined prompt from the LangChain hub

# Define the LLM object using OpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
# Initialize the language model (LLM) with OpenAI's GPT-3.5-turbo with a temperature of 0 for deterministic responses

# The function to combine multiple document into one
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
# Define a function to concatenate the content of multiple documents into a single string

# Define whole chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
# Define the Retrieval-Augmented Generation (RAG) chain:
# 1. Retrieve relevant document chunks and format them
# 2. Combine the formatted context with the question
# 3. Pass the combined input to the language model
# 4. Parse the output string

result = rag_chain.invoke("What is attention?")
print(result)
# Invoke the RAG chain with the question "What is attention?" and print the result

In [16]:
# Import necessary modules
import nltk
# nltk.download('averaged_perceptron_tagger')

# Loader for multiple text files
loader = DirectoryLoader('/Users/junjiezhang/Documents/GitHub/LLM_Zeng24/backend_langchain', glob="*.txt")
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

# Create embeddings for each chunk
vectorstore = Chroma.from_documents(chunks, OpenAIEmbeddings())

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


In [17]:
import json
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the API key
LANGSMITH_API_KEY = os.getenv('LANGSMITH_API_KEY')


# Define a retriever from the vector DB
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

# Get a prompt from LangChain hub
prompt = hub.pull("rlm/rag-prompt")

In [18]:
# Define the LLM object using OpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


In [19]:
# The function to combine multiple document into one
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [20]:
# Define whole chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [21]:
import textwrap

result = rag_chain.invoke("What is attention?")
wrapped_result = textwrap.fill(result, width=80)
print(wrapped_result)

Attention is a mechanism that maps a query and key-value pairs to an output
using vectors. The output is a weighted sum of values based on the compatibility
of the query with the corresponding key. Self-attention is an example of
attention used in various tasks like reading comprehension and summarization.


## test 01: chat

In [23]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

from langchain_core.messages import HumanMessage, SystemMessage


llm1 = ChatOpenAI()
llm2 = ChatAnthropic(model_name="claude-2.0")



In [24]:
# Define the messages for the conversation
messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content="Suggest some good names for a shoe company."),
]
# The SystemMessage sets the context for the assistant
# The HumanMessage asks the assistant to suggest names for a shoe company

# Invoke the first language model (llm1) with the messages
result1 = llm1.invoke(messages)
print("Result from OpenAI:", result1.content)
# Print the result from the OpenAI model




Result from OpenAI: 1. SoleStride Footwear
2. StepUp Shoes
3. Peak Performance Footwear
4. StrideTrend Shoes
5. Trek & Tread Shoes
6. UrbanWalk Footwear
7. SwiftStride Shoes
8. BoldStep Footwear
9. VentureWalk Shoes
10. SteadyStride Footwear


In [31]:
# Load environment variables from .env file
load_dotenv()

# Get the API key for Anthropic
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')



In [32]:
# Set the API key for the Anthropic model
llm2 = ChatAnthropic(model_name="claude-2.0", api_key=ANTHROPIC_API_KEY)

# Invoke the second language model (llm2) with the messages
result2 = llm2.invoke(messages)
print("Result from Anthropic:", result2.content)
# Print the result from the Anthropic model


BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits.'}}

In [33]:
# Stream results from the first language model (llm1)
print("Streaming result from OpenAI:")
for chunk in llm1.stream(messages):
    print(chunk.content, end="", flush=True)
# Print each chunk of the response from the OpenAI model in real-time

print("\n")



Streaming result from OpenAI:
1. Soles and Strides
2. Footwear Fusion
3. StepStar
4. WalkWise
5. HappyFeet Co.
6. UrbanKick
7. SoleHaven
8. StrideStyle
9. Footworks Studio
10. Steppin' Up Shoes



In [34]:
# Stream results from the second language model (llm2)
print("Streaming result from Anthropic:")
for chunk in llm2.stream(messages):
    print(chunk.content, end="", flush=True)
# Print each chunk of the response from the Anthropic model in real-time

print("\n")

Streaming result from Anthropic:


BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits.'}}

## test 2: chat with promote


In [37]:
# Import necessary classes from langchain_openai and langchain.schema
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# Define the system message template for setting the context
system_template = "You are a helpful assistant that translates {input_language} to {output_language}."

# Define the human message template for the input text
human_template = "{text}"

# Create the final prompt template using the system and human templates
final_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),  # System message placeholder
    ("human", human_template),    # Human message placeholder
])

# Uncomment the following lines to see the formatted prompt
print(final_prompt.format(
    input_language="English", 
    output_language="French", 
    text="I love programming.",
))



System: You are a helpful assistant that translates English to French.
Human: I love programming.


In [42]:
# Initialize the chat model
chat_model = ChatOpenAI()

# Create a chain by combining the final prompt and the chat model
chain = final_prompt | chat_model

# Invoke the chain with specific input parameters
result = chain.invoke(dict(
    input_language="Chinese", 
    output_language="English", 
    text="飞流直下三千尺，疑是银河落九天。",
))

# Print the result content
print(result.content)

The waterfall cascades down three thousand feet, resembling the Milky Way plunging from the ninth heaven.


## test 3: chat with few shot

In [49]:
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

# Define examples for few-shot learning
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
]

# Create a prompt template for the examples
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),  # Human message placeholder
        ("ai", "{output}"),    # AI response placeholder
    ]
)

# Create a few-shot prompt template using the examples
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# Create the final prompt template
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),  # System message setting the context
        few_shot_prompt,  # Include the few-shot examples
        ("human", "{input}"),  # Placeholder for the human input
    ]
)

# Initialize the language model with a temperature setting
llm = ChatOpenAI(temperature=0)

# Create a chain by combining the final prompt and the language model
chain = final_prompt | llm

# Invoke the chain with a specific input
result = chain.invoke({"input": "how to say hello in chinese?"})

# Print the result content
print(result.content)


你好 (nǐ hǎo)


## test 4: output parser

In [51]:
# Import necessary classes from langchain_openai, langchain.prompts, and langchain.schema
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [52]:


# Define a custom output parser class to parse the output into a comma-separated list
class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

# Define the system message template for setting the context
template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""

# Define the human message template for the input text
human_template = "{text}"

# Create the final prompt template using the system and human templates
final_prompt = ChatPromptTemplate.from_messages([
    ("system", template),  # System message placeholder
    ("human", human_template),  # Human message placeholder
])

# Create a chain by combining the final prompt, the chat model, and the custom output parser
chain = final_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()

# Invoke the chain with a specific input
result = chain.invoke({"text": "colors"})

# Print the parsed result
print(result)

['red', 'blue', 'green', 'yellow', 'purple']


## test 05 RAG

In [53]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.vectorstores import FAISS

In [54]:
# Define a list of text data
text_list = [
    "Johnson worked at MS",
    "Harry worked at Meta",
    "Hua worked at Alibaba",
    "Harrison worked at Kensho",
]

# Create a vector store from the text data using OpenAI embeddings
vectorstore = FAISS.from_texts(text_list, embedding=OpenAIEmbeddings())

# Create a retriever from the vector store
retriever = vectorstore.as_retriever()

# Define the prompt template for the model
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Initialize the chat model
model = ChatOpenAI()

# Create a chain by combining the retriever, prompt, model, and output parser
chain = (
    {"context": retriever, "question": RunnablePassthrough()}  # Pass the retriever and question
    | prompt  # Apply the prompt template
    | model  # Use the chat model
    | StrOutputParser()  # Parse the output as a string
)

# Invoke the chain with a specific question
result = chain.invoke("where did Harrison work?")

# Print the result
print(result)

Harrison worked at Kensho.


## test 6 chat with memory

In [55]:
from langchain.prompts import (
    ChatPromptTemplate, 
    MessagesPlaceholder, 
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate
)
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory

In [57]:
# Initialize the chat model
llm = ChatOpenAI()

# Create the prompt template for the conversation
prompt = ChatPromptTemplate(
    messages=[
        # System message to set the context for the chatbot
        SystemMessagePromptTemplate.from_template("You are a nice chatbot having a conversation with a human."),
        # Placeholder for the chat history to maintain the conversation context
        MessagesPlaceholder(variable_name="chat_history"),
        # Template for the human message input
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

# Create a memory object to store the conversation history
# `return_messages=True` ensures that the memory returns messages to fit into the MessagesPlaceholder
# `memory_key="chat_history"` aligns with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Create a conversation chain by combining the chat model, prompt, and memory
conversation = LLMChain(
    llm=llm,  # The chat model
    prompt=prompt,  # The prompt template
    verbose=True,  # Enable verbose mode to see detailed logs
    memory=memory  # The memory object to store conversation history
)



In [58]:
# Invoke the conversation chain with a specific question
# The `chat_history` gets populated by the memory automatically
result = conversation.invoke({"question": "Hi, my name is Huajun."})
print(result["text"])  # Print the chatbot's response





> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: Hi, my name is Huajun.

> Finished chain.
Hello Huajun! It's nice to meet you. How are you today?


In [59]:
# Invoke the conversation chain with another question
result = conversation.invoke({"question": "Write a poem for me."})
print(result["text"])  # Print the chatbot's response



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: Hi, my name is Huajun.
AI: Hello Huajun! It's nice to meet you. How are you today?
Human: Write a poem for me.

> Finished chain.
Of course, Huajun! Here's a poem just for you:

In a world of beauty, so vast and wide,
Huajun stands with joy and pride.
With a heart that's kind and true,
Spreading love in all you do.

May your days be filled with light,
Guiding you through the darkest night.
So let this poem be a token,
Of the friendship that we've awoken.


## test 7 chat with memory rag continous

In [60]:
from langchain.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [62]:
print("Loading documents")
# Load documents from the local disk using a directory loader
loader = DirectoryLoader('./', glob="*.txt")
documents = loader.load()

# Split documents into smaller chunks for better processing
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks = text_splitter.split_documents(documents)

print("Creating embeddings")

# Create embeddings for each chunk using OpenAI embeddings
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(chunks, embeddings)

# Create a retriever from the vector store to fetch relevant chunks
retriever = vectorstore.as_retriever()

print("Creating chains")


libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


Loading documents


libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


Creating embeddings
Creating chains


In [64]:
# Initialize the chat model
llm = ChatOpenAI()

# Create a memory object to store the conversation history
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Create a conversational retrieval chain by combining the chat model, retriever, and memory
conversation = ConversationalRetrievalChain.from_llm(
    llm, retriever=retriever, memory=memory, verbose=True
)

# Start an infinite loop to interact with the user
# while(True):
#     user_input = input("> ")  # Get user input
#     result = conversation.invoke(user_input)  # Invoke the conversation chain with the user input
#     print(result["answer"])  # Print the chatbot's response


# Start an infinite loop to interact with the user
# Start an infinite loop to interact with the user
while True:
    user_input = input("> ")  # Get user input
    if user_input.lower() == "exit":  # Check if the user wants to exit
        print("Exiting the conversation.")
        break  # Exit the loop
    result = conversation.invoke(user_input)  # Invoke the conversation chain with the user input
    print(result["answer"])  # Print the chatbot's response



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
computing:

I. Introduction, his life and background

A: His early life

B:

rewrite Covert my resume into a profile overview.

{resume}

Profile overview:

rewrite Rephrase this for me: "I can’t seem to find out how to work this darn thing."

Alternate phrasing: "

rewrite Original: She no go to sleep.

Standard American English: She didn’t go to sleep

Original: It real bad for I to make do of this.

Standard American English:

chat The following is a conversation with an AI assistant. The assistant is helpful,

creative, clever, and very friendly.

Human: Hello, who are you?

AI: I am an AI created by OpenAI. How can I help you today?

Human: I’m feeling kind of down today.

AI:

Continued on next

## test 8 Pinecone

In [65]:
# Import necessary classes from pinecone, langchain_openai, and langchain_pinecone
from pinecone import Pinecone
pc = Pinecone()

# Define the index name and initialize the index
index_name = 'research-paper-rag-index'
index = pc.Index(index_name)

# Describe the index statistics
index.describe_index_stats()

# Import OpenAI embeddings
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Import Pinecone vector store
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

# Import uuid for generating unique IDs
from uuid import uuid4

# Import document loader and text splitter
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load documents from the local directory
loader = DirectoryLoader('./', glob="*.txt")
documents = loader.load()

# Split documents into smaller chunks for better processing
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
document_chunks = text_splitter.split_documents(documents)

# Generate unique IDs for each document chunk
uuids = [str(uuid4()) for _ in range(len(document_chunks))]

# Add document chunks to the vector store with the generated IDs
vector_store.add_documents(documents=document_chunks, ids=uuids)

PineconeConfigurationError: You haven't specified an Api-Key.